### Part 1

In [131]:
from copy import deepcopy

file = open("../Data/input_03.txt", "r")
content = file.read().split('\n')[:-1]
file.close()


def find_symbol_number_masks(content, search_symbols, numbers):

    # create a symbol mask from map  [search symbol] -> 1  [. 0-9] -> 0
    symbol_mask = []
    # create a number mask from map  [0-9] -> 1  [otherwise] -> 0
    number_mask = []

    for line in content:
        symbol_mask_line = []
        number_mask_line = []

        for c in list(line):

            if c in search_symbols:
                symbol_mask_line.append(1)
                number_mask_line.append(0)

            elif c in numbers:
                symbol_mask_line.append(0)
                number_mask_line.append(1)

            else:
                symbol_mask_line.append(0)
                number_mask_line.append(0)

        symbol_mask.append(symbol_mask_line)
        number_mask.append(number_mask_line)

    return symbol_mask, number_mask
    

def mask_expansion(mask, i, j, last_index):

    mask[i][j] = 1

    # not first element
    if j != 0:
        mask[i][j-1] = 1

    # not last element
    if j != last_index:
        mask[i][j+1] = 1

    return mask


def expand_the_mask(symbol_mask):

    symbol_mask_exp = deepcopy(symbol_mask)

    for i, line_mask in enumerate(symbol_mask):
        
        for j, el in enumerate(line_mask):
            
            last_index = len(line_mask)-1

            if el == 1:
                
                symbol_mask_exp = mask_expansion(symbol_mask_exp, i, j, last_index)

                # not first line
                if i != 0:
                    symbol_mask_exp = mask_expansion(symbol_mask_exp, i-1, j, last_index)

                # not last line
                if i != 0:
                    symbol_mask_exp = mask_expansion(symbol_mask_exp, i+1, j, last_index)

    return symbol_mask_exp


def index_mask(number_mask):

    # Go through a make an index mapping of number 
    # [. . . 9 9 8 . . 3 3 . . 8 3 1 . .]  >>  [. . . 1 1 1 . . 2 2 . . 3 3 3 . .]
    idx_mask = deepcopy(number_mask)
    counter = 1
    number_dict = {}

    for i, number_line in enumerate(number_mask):

        element = 0
        last_index = len(number_line)-1
        
        number = []

        while element < last_index+1:
            
            if number_line[element]==1:
            
                idx_mask[i][element] = counter
                number.append(content[i][element])

                if (element == last_index):
                    number_dict[counter] = int(''.join(number))
                    number = []
                    counter += 1

                elif idx_mask[i][element+1] == 0:
                    number_dict[counter] = int(''.join(number))
                    number = []
                    counter += 1

            element += 1

    return idx_mask, number_dict


def unique_adjacent_indices(symbol_mask_exp, idx_mask):

    # Finding indices of number that are adjacent to a symbol
    connected = []
    for i, line_mask in enumerate(symbol_mask_exp):
        
        for j, el in enumerate(line_mask):
            if el == 1:
                connected.append(idx_mask[i][j])

    
    # set to remove duplicates of indices and remove 0
    connected_indices = list(set(connected).symmetric_difference([0]))

    return connected_indices


# relavent symbols
search_symbols = ['*', '=', '/', '%', '#', '&', '$', '-', '@', '+']
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

symbol_mask, number_mask = find_symbol_number_masks(content, search_symbols, numbers)

symbol_mask_exp = expand_the_mask(symbol_mask)

idx_mask, number_dict = index_mask(number_mask)

connected_indices = unique_adjacent_indices(symbol_mask_exp, idx_mask)


# Find sum of all numbers from number_dict (that connects index to numbers)
sum = 0

for ci in connected_indices:
   sum += number_dict[ci]

print('Sum of numbers adjacent to symbols: ', sum)


Sum of numbers adjacent to symbols:  557705


### Part 2

In [151]:
# find the expanded mask around each * 
# check if two different indices in idx_mask are hit by it
# if so add pair to lst-lst

def check_sides_4_hits(hits, i, j, idx_mask):
            
    hits.append(idx_mask[i][j])

     # not first element
    if j != 0:
        hits.append(idx_mask[i][j-1])

    # not last element
    if j != last_index:
        hits.append(idx_mask[i][j+1])

    return hits


def find_double_overlaps(symbol_mask, idx_mask):

    hits_lst = []

    for i, line_mask in enumerate(symbol_mask):
        for j, el in enumerate(line_mask):
            
            last_index = len(line_mask)-1
            
            hits = []

            if el == 1:

                hits = check_sides_4_hits(hits, i, j, idx_mask)

                # not first line
                if i != 0:
                    hits = check_sides_4_hits(hits, i-1, j, idx_mask)

                # not last line
                if i != 0:
                    hits = check_sides_4_hits(hits, i+1, j, idx_mask)

                
                hits_lst.append( list(set(hits).symmetric_difference([0])) )

    return hits_lst


search_symbols = ['*']
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

symbol_mask, number_mask = find_symbol_number_masks(content, search_symbols, numbers)
idx_mask = index_mask(number_mask)

symbol_mask_exp = deepcopy(symbol_mask)

hits_lst = find_double_overlaps(symbol_mask, idx_mask)

sum = 0 

for h in hits_lst:
    if len(h) == 2:
        sum += number_dict[h[0]] * number_dict[h[1]]

print('Sum of each multiplication pair of numbers adjacent to * symbol: ', sum)


Sum of each pair of numbers adjacent to a * symbol:  84266818
